<h4 >Mahdi Abbariki</h4>
<h5>983212001</h5>

objective: <br>
    minimizing equation using ES algorithm

First we import methods and libraries we need

In [141]:
from numpy import power
from numpy import sum
from numpy.random import uniform
from numpy.random import normal
from numpy import argsort


define a method to evaluate equation

In [142]:
def objective(x,n):
    powers = power(x[:n], 2)
    return sum(powers)

define a method to check if a generated candidate is in the specified bounds or not

In [143]:
def in_bounds(point, bounds):
    for i in range(len(point)):
        item = point[i]
        if item < bounds[0] or item > bounds[1]:
            return False
    return True

create initial population

In [144]:
def initPopulation(lam,bounds,n):
    population = list()
    for _ in range(lam):
        candidate = None
        while candidate is None or not in_bounds(candidate, bounds):
            candidate = []
            for _ in range(n):
                candidate.append(uniform(low=bounds[0], high=bounds[1]))
        population.append(candidate)
    return population

now we can create the esPlus method to deal with searching

In [145]:
def esPlus(mu, lam, maxIter, objective, bounds, n, population):
    best, best_eval = None, 1e+100
    n_children = int(lam / mu)
    normalScale = 1
    normalScaleC = 0.9

    # perform the search
    for epoch in range(maxIter):
	    # evaluate fitness for the population
        scores = [objective(c,n) for c in population]
        # rank scores in ascending order
        ranks = argsort(argsort(scores))
        # select the indexes for the top mu ranked solutions
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]

        #create children
        children = []
        for i in selected:
            #update best and best eval based on parent
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
           
            # keep the parent
            children.append(population[i])
            #count better children for calculating normalScare dynamically
            betterChild = 0
            # create children for parent
            for _ in range(n_children):
                child = None
                while child is None or not in_bounds(child,bounds):
                    child = []
                    for j in range(n):
                        child.append(population[i][j] + normal(loc=0, scale=normalScale))
                if(objective(child,n) < objective(population[i],n)): # child is better than parent
                    betterChild += 1
                children.append(child)
        ps = betterChild / n_children

        if(ps < 1/5):
            normalScale *= normalScaleC
        elif(ps > 1/5):
            normalScale /= normalScaleC

        # print(normalScale,ps,betterChild)

        # replace current populations with children
        population = children
    return [best,best_eval]

create es comma algorithm as well

In [146]:
def esComma(mu,lam,maxIter,objective,bounds,n,population):
    best, best_eval = None, 1e+100
    n_children = int(lam / mu)
    normalScale = 1
    normalScaleC = 0.9

    # perform the search
    for epoch in range(maxIter):
	    # evaluate fitness for the population
        scores = [objective(c,n) for c in population]
        # rank scores in ascending order
        ranks = argsort(argsort(scores))
        # select the indexes for the top mu ranked solutions
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]

        #create children
        children = []
        for i in selected:
            #update best and best eval based on parent
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
           
            #count better children for calculating normalScare dynamically
            betterChild = 0
            # create children for parent
            for _ in range(n_children):
                child = None
                while child is None or not in_bounds(child,bounds):
                    child = []
                    for j in range(n):
                        child.append(population[i][j] + normal(loc=0, scale=normalScale))
                if(objective(child,n) < objective(population[i],n)): # child is better than parent
                    betterChild += 1
                children.append(child)
        
        # evaluate new scale for gaussian distribution
        ps = betterChild / n_children
        if (ps < 1/5):
            normalScale *= normalScaleC
        elif (ps > 1/5):
            normalScale /= normalScaleC

        # replace current populations with children
        population = children
    return [best,best_eval]

specify the inputs and run the algorithm

In [147]:
bounds = [-100,100]

maxIterations = 5000

mu = 2

lam = 4

n = 100

population = initPopulation(lam, bounds, n) # same initial population for two algorithm

best, score = esPlus(mu,lam,maxIterations,objective,bounds,n,population)
print('ES Plus Done!')
print('f(%s) = %f' % (best, score))

print("---------------------------")

best, score = esComma(mu, lam, maxIterations, objective, bounds, n,population)
print('ES Comma Done!')
print('f(%s) = %f' % (best, score))


ES Plus Done!
f([5.665823982973315e-10, -1.5146649596651221e-09, -3.6332982737511355e-10, -7.761316395998634e-10, 2.225210119647783e-09, -5.25209952006054e-10, -7.689962305892046e-10, 8.192967914917064e-10, -1.4930513358804363e-09, -1.9613121025716496e-10, -1.1989635429556062e-10, 1.0097702922847483e-09, 1.343918590308935e-10, -4.948499742906742e-10, 6.101929415574262e-10, -9.853893436501818e-10, 7.785227749991018e-10, -9.688786176993563e-10, -1.2876051518804698e-09, 1.4950352072703534e-09, -7.109800269304237e-10, 7.028402527094857e-11, -1.0026962027612307e-09, -2.00293505254549e-09, 1.9815853413796387e-09, -1.5724445280061143e-09, -6.026456551428782e-10, -1.3022849375378298e-09, -2.015223677795709e-09, -3.1758908839183016e-10, 8.086886945962161e-10, -1.947562709623872e-09, 1.894933284847205e-09, 1.2429819959236982e-09, 4.622598081660163e-11, -8.068172507244836e-11, 4.0272334862396366e-10, -1.6029483138494412e-09, 1.0445977278577616e-09, -8.009164742906713e-10, 9.236812644093042e-10, -